## Creates the table where the predictions will be store

Fri 1 Aug 2025: I'm deprecating the event_detections and plate_recognitions table so all predictions are stored here from now on.

In [ ]:
import boto3
from botocore.exceptions import ClientError

dynamodb = boto3.resource("dynamodb", region_name="eu-west-1")
TABLE_NAME = "event_predictions"

try:
    table = dynamodb.create_table(
        TableName=TABLE_NAME,
        KeySchema=[
            {"AttributeName": "device_id", "KeyType": "HASH"},
            {"AttributeName": "frame_timestamp", "KeyType": "RANGE"},
        ],
        AttributeDefinitions=[
            {"AttributeName": "device_id", "AttributeType": "S"},
            {"AttributeName": "frame_timestamp", "AttributeType": "S"},
            {"AttributeName": "event_timestamp", "AttributeType": "S"},  # for GSI
        ],
        GlobalSecondaryIndexes=[
            {
                "IndexName": "event_timestamp_index",
                "KeySchema": [
                    {"AttributeName": "event_timestamp", "KeyType": "HASH"},
                    {"AttributeName": "device_id", "KeyType": "RANGE"},
                ],
                "Projection": {"ProjectionType": "ALL"},
            }
        ],
        BillingMode="PAY_PER_REQUEST"
    )
    print("Creating table...")
    table.wait_until_exists()
    print(f"Table '{TABLE_NAME}' created.")
except ClientError as e:
    if e.response["Error"]["Code"] == "ResourceInUseException":
        print(f"Table '{TABLE_NAME}' already exists.")
    else:
        raise